In [3]:
import grpc
import json
import routing_pb2
import routing_pb2_grpc

def test_routing():
    # Connect to gRPC server
    with grpc.insecure_channel('localhost:50051') as channel:
        stub = routing_pb2_grpc.RoutingServiceStub(channel)
        
        # Test health check first
        try:
            health_response = stub.HealthCheck(routing_pb2.HealthRequest())
            print(f"Health Check: {health_response.status} - {health_response.message}")
        except Exception as e:
            print(f"Health check failed: {e}")
            return
        
        # Test routing
        try:
            request = routing_pb2.RouteRequest(
                start_lat=31.2235494,
                start_lon=29.9373941,
                end_lat=31.193363,
                end_lon=29.9067595,
                max_transfers=2,
                walking_cutoff=1000.0,
                restricted_modes=["Bus", "Minibus"],
                weights = {
                    "time": 0.5,
                    "cost": 0.3,
                    "walk": 0.1,
                    "transfer": 0.1
                },
                top_k=10
            )
            
            print("\nSending routing request...")
            response = stub.FindRoute(request)
            
            # Parse the JSON response
            journey_data = json.loads(response.journeys_json)
            # save to a file for inspection
            with open("journey_data.json", "w") as f:
                json.dump(journey_data, f, indent=4)
            print("Routing response saved to journey_data.json")
        except Exception as e:
            print(f"Routing request failed: {e}")

if __name__ == "__main__":
    test_routing()

Health Check: healthy - Transit Routing gRPC Service is running

Sending routing request...
Routing response saved to journey_data.json


In [11]:
!python -c "import routing_pb2; print(dir(routing_pb2.RouteRequest))"

['ByteSize', 'Clear', 'ClearExtension', 'ClearField', 'CopyFrom', 'DESCRIPTOR', 'DiscardUnknownFields', 'Extensions', 'FindInitializationErrors', 'FromString', 'HasExtension', 'HasField', 'IsInitialized', 'ListFields', 'MergeFrom', 'MergeFromString', 'ParseFromString', 'SerializePartialToString', 'SerializeToString', 'SetInParent', 'UnknownFields', 'WhichOneof', '_CheckCalledFromGeneratedFile', '_ListFieldsItemKey', '_SetListener', '__class__', '__contains__', '__deepcopy__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__unicode__']
